In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pioç
from statsmodels.tsa.stattools import pacf
from sklearn.base import TransformerMixin
from skits.preprocessing import HorizonTransformer
import datetime
from sktime.datasets import load_airline
from sktime.forecasting.compose import ReducedForecaster
from sktime.forecasting.model_selection import temporal_train_test_split, SlidingWindowSplitter
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
from collections import defaultdict
from sklearn.feature_selection import SelectKBest, mutual_info_regression
import lightgbm as lgb

C:\Users\vpinilla\AppData\Local\Continuum\miniconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\vpinilla\AppData\Local\Continuum\miniconda3\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\vpinilla\AppData\Local\Continuum\miniconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\vpinilla\AppData\Local\Continuum\miniconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\Users\vpinilla\AppData\Local\Continuum\miniconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
points = 50
window_length = 100
rolling_window = [5,10,20]
horizon = 1
step = 1
freq = '1T'

In [3]:
df = pd.read_csv('Seat.csv')
# df = df.drop([5441,5445,50241,50535,50926,51490,52196,52418,54124,58615,58628,58676,59132,60029,60693,64036,65419])
df.index = pd.date_range(start='2000-02-05 14:07:33.511000+00:00', periods=len(df), 
                         freq=freq).tz_convert('Europe/Berlin')
df = df.iloc[:4000,[10]]
df

,INSTALACIONES [kWh]
2000-02-05 15:07:33.511000+01:00,429.136414
2000-02-05 15:08:33.511000+01:00,441.115204
2000-02-05 15:09:33.511000+01:00,437.226990
2000-02-05 15:10:33.511000+01:00,415.408021
2000-02-05 15:11:33.511000+01:00,432.827713
...,...
2000-02-08 09:42:33.511000+01:00,455.932999
2000-02-08 09:43:33.511000+01:00,481.249069
2000-02-08 09:44:33.511000+01:00,462.117005
2000-02-08 09:45:33.511000+01:00,478.533279


In [4]:
y_train, y_test = train_test_split(df, test_size=points, shuffle=False)
print(y_train.shape[0], y_test.shape[0])

3950 50


-----

#### Create train features

In [5]:
from extract_sample_features import *

In [6]:
def create_sample_features(X, window_length = 5, features = [], rolling_window = [2]):
    lags_X = lags_sample(X, window_length=window_length, step=1)
    df = lags_X.copy()
    for f in features:
        if rolling_window:
            for rol in rolling_window:
                aux = switch_sample_features(f, X, lags_X, rol)
                df = pd.concat([df, aux], axis = 1).dropna()
        else:
            aux = switch_sample_features(f, X, lags_X, [])
            df = pd.concat([df, aux], axis = 1).dropna()
    return df

In [7]:
def switch_sample_features(value, X, lags_X, rolling_window):
    return { 
        'mean': lambda : mean_sample(lags_X, rolling_window),
        'std': lambda : std_sample(lags_X, rolling_window),
        'max': lambda : max_sample(lags_X, rolling_window),
        'min': lambda : min_sample(lags_X, rolling_window),
        'quantile': lambda : quantile_sample(lags_X, rolling_window),
        'iqr': lambda : iqr_sample(lags_X, rolling_window),
        'entropy': lambda : entropy_sample(lags_X, rolling_window),
        'trimmean': lambda : trimmean_sample(lags_X, rolling_window),
        'variation': lambda : variation_sample(lags_X, rolling_window),
        'hmean': lambda : hmean_sample(lags_X, rolling_window),
        'gmean': lambda : gmean_sample(lags_X, rolling_window),
        'mad': lambda : mad_sample(lags_X, rolling_window),
#         'gstd': lambda : gstd_sample(lags_X, rolling_window),
        'tvar': lambda : tvar_sample(lags_X, rolling_window),
        'kurtosis': lambda : kurtosis_sample(lags_X, rolling_window),
        'sem': lambda : sem_sample(lags_X, rolling_window),
        'wav': lambda : wav_sample(lags_X, rolling_window),
        #-----------------------------------------
        'minute': lambda : minute_sample(X),
        'hour': lambda : hour_sample(X),
        'dayofweek': lambda : dayofweek_sample(X),
        'day': lambda : day_sample(X),
        'month': lambda : month_sample(X),
        'quarter': lambda : quarter_sample(X),
        'weekofyear': lambda : weekofyear_sample(X),
        'weekend': lambda : weekend_sample(X)
        
    }.get(value)()

#### Create horizon

In [8]:
def create_horizon(df, horizon):
    y = df.copy()
    ht = HorizonTransformer(horizon = horizon+1)
    y_horizon = ht.fit_transform(y.iloc[:,0]) #Los horizon ultimos valores son nan ya que no se puede crear un horizonte completo para ellos
    y_horizon = pd.DataFrame(y_horizon[:-horizon, :], index = y.index[horizon:])
    name = 't+'+str(horizon)
    y_horizon = pd.DataFrame({name : y_horizon.iloc[:, -1]})
    
    return y_horizon

#### Feature selection

In [9]:
def feature_selection(X, y, num_features):
    clf = lgb.LGBMRegressor(n_estimators=40).fit(X, y)
    best_indx_col = clf.feature_importances_.argsort()[-num_features:]
    best_features = list(X_train.columns[best_indx_col])
    return order_by_other_list(X_train.columns, best_features)

------

In [10]:
def order_by_other_list(list_order, list_tobe_order):
    d = {k:v for v,k in enumerate(list_order)}
    list_tobe_order.sort(key=d.get)
    return list_tobe_order

----

In [11]:
from extract_features import *

In [12]:
def switch_features(value, lags_data, date, rolling_window, num):
    return {
        'lag': lambda : lag_feature(lags_data, num),
        #--------------------------------------------------------------
        'mean': lambda : mean_feature(lags_data, rolling_window, num),
        'std': lambda : std_feature(lags_data, rolling_window, num),
        'max': lambda : max_feature(lags_data, rolling_window, num),
        'min': lambda : min_feature(lags_data, rolling_window, num),
        'quantile': lambda : quantile_feature(lags_data, rolling_window, num),
        'iqr': lambda : iqr_feature(lags_data, rolling_window, num),
        'entropy': lambda : entropy_feature(lags_data, rolling_window, num),
        'trimmean': lambda : trimmean_feature(lags_data, rolling_window, num),
        'variation': lambda : variation_feature(lags_data, rolling_window, num),
        'hmean': lambda : hmean_feature(lags_data, rolling_window, num),
        'gmean': lambda : gmean_feature(lags_data, rolling_window, num),
        'mad': lambda : mad_feature(lags_data, rolling_window, num),
#         'gstd': lambda : gstd_feature(lags_data, rolling_window, num),
        'tvar': lambda : tvar_feature(lags_data, rolling_window, num),
        'kurtosis': lambda : kurtosis_feature(lags_data, rolling_window, num),
        'sem': lambda : sem_feature(lags_data, rolling_window, num),
        'wav': lambda : wav_feature(lags_data, rolling_window, num),
        #--------------------------------------------------------------
        'minute': lambda : minute_feature(date, num),
        'hour': lambda : hour_feature(date, num),
        'dayofweek': lambda : dayofweek_feature(date, num),
        'day': lambda : day_feature(date, num),
        'month': lambda : month_feature(date, num),
        'quarter': lambda : quarter_feature(date, num),
        'weekofyear': lambda : weekofyear_feature(date, num),
        'weekend': lambda : weekend_feature(date, num)
    }.get(value)()

In [13]:
def create_features(feature_names, lags_data, date):
    features = []
    for f in feature_names:
        ftype, info = f.split('_',1)
        if '_' in info:
            rolling_window, num = info.split('_')
            num = int(num)
            rolling_window = int(rolling_window)
        else:
            num = int(info)
            rolling_window = None
        features.append(switch_features(ftype, lags_data, date, rolling_window, num))
                                
    return features

---

In [14]:
def switch_optimization(value, regressor):
    return {
        'tpe': lambda : tpe_optimization(),
        'pso': lambda : pso_optimization(),
    }.get(value)()

---

In [15]:
def recursive_forecast(y, model, window_length, feature_names, rolling_window, n_steps):
    """
    Parameters
    ----------
    y: pd.Series holding the input time-series to forecast
    model: pre-trained machine learning model
    lags: list of lags used for training the model
    n_steps: number of time periods in the forecasting horizon
    step: forecasting time period
   
    Returns
    -------
    fcast_values: pd.Series with forecasted values
    """
   
    # get the dates to forecast
    last_date = y.index[-1] + datetime.timedelta(minutes=15)
    target_range = pd.date_range(last_date, periods=n_steps, freq=freq)
    target_value = np.arange(n_steps)
    max_rol = max(rolling_window, default=1)
    lags = list(y.iloc[-(window_length+(max_rol-1)):,0].values)
    ####
    
    
    for i in range(n_steps):
        train = create_features(feature_names, lags, target_range[i])
        new_value = model.predict(pd.DataFrame(train).transpose())
        target_value[i] = new_value[0]
        lags.pop(0)
        lags.append(new_value[0])
           
    return target_value

---

In [16]:
# features = ['mean', 'std', 'max', 'min', 'quantile', 'iqr', 'entropy', 'trimmean', 'variation', 'hmean', 'gmean', 'mad', 'tvar',
#             'kurtosis', 'sem', 'minute', 'hour', 'dayofweek', 'day', 'month', 'quarter', 'weekofyear', 'weekend']
features = ['mean', 'std', 'max', 'min', 'minute']

In [17]:
X_train = create_sample_features(y_train, window_length=window_length, features=features, rolling_window=rolling_window)
y_horizon = create_horizon(y_train, horizon)
y_horizon = y_horizon.loc[X_train.index[0]:,:]
X_train.head()

,lag_100,lag_99,lag_98,lag_97,lag_96,lag_95,lag_94,lag_93,lag_92,lag_91,...,minute_51,minute_52,minute_53,minute_54,minute_55,minute_56,minute_57,minute_58,minute_59,minute_60
2000-02-05 17:06:33.511000+01:00,441.426468,420.912125,363.387573,388.791611,404.939103,423.678970,429.227005,415.894593,426.114227,425.496506,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-02-05 17:07:33.511000+01:00,420.912125,363.387573,388.791611,404.939103,423.678970,429.227005,415.894593,426.114227,425.496506,427.149979,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-02-05 17:08:33.511000+01:00,363.387573,388.791611,404.939103,423.678970,429.227005,415.894593,426.114227,425.496506,427.149979,413.574082,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-02-05 17:09:33.511000+01:00,388.791611,404.939103,423.678970,429.227005,415.894593,426.114227,425.496506,427.149979,413.574082,337.172134,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-02-05 17:10:33.511000+01:00,404.939103,423.678970,429.227005,415.894593,426.114227,425.496506,427.149979,413.574082,337.172134,413.618301,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**De 15 lags seleccionamos los 5 primeros**

In [ ]:
selected_feat = 100
best_features = feature_selection(X_train, y_horizon.values.ravel(), selected_feat)

In [ ]:
X_train_selec = X_train.loc[:, best_features]
X_train_selec.head()

In [ ]:
regressor = lgb.LGBMRegressor()
regressor.fit(X_train_selec, y_horizon.values.ravel())

In [ ]:
pred_manual = recursive_forecast(y_train, regressor, window_length=window_length, feature_names=best_features,
                                rolling_window=rolling_window, n_steps=points)
print('RMSE loss: %.2f' % (mean_squared_error(y_test, pred_manual, squared=False)))

In [ ]:
fig = go.Figure()
# fig.add_trace(
#     go.Scatter(x=list(y_train.index), y=list(y_train.iloc[:,0]),mode='lines+markers',name = "Train"))
fig.add_trace(
    go.Scatter(x=list(y_test.index), y=list(y_test.iloc[:,0]), mode='lines+markers', name = "Test"))
fig.add_trace(
    go.Scatter(x=list(y_test.index), y=list(pred_manual), line=go.scatter.Line(color="green"), name = "Pred"))
fig.show()

----

In [ ]:
def check_well(y, X, window_length, feature_names, n_steps, rolling_window=[]):   
    target_range = pd.date_range(X.index[0], periods=n_steps, freq=freq)
    names = [i.split('_')[0] for i in feature_names]
    max_rol = max(rolling_window, default=1)
    lags = list(y.iloc[:window_length+max_rol-1,0])
    for i in range(n_steps):
        train = create_features(feature_names, lags, target_range[i])
        train = [round(num, 2) for num in train]
        r = [round(num, 2) for num in list(X.iloc[i,:])]
        if not r == train:
            print('Not all features have been created correctly')
            return
        lags.pop(0)
        lags.append(float(y.iloc[window_length+max_rol-1+i,:]))
    print('All features have been created correctly')
    return 

In [ ]:
check_well(y_train, X_train_selec, window_length, best_features ,points, rolling_window)